In [1]:
import pandas as pd

In [2]:
raw = pd.read_csv("data1.csv")
raw.report_text[0]

'\n\nRecord date: 2106-02-12\n\nCampbell Orthopedic Associates\n4 Madera Circle\nOmak, GA 28172\n \nHabib Valenzuela, M.D.\n \n \n                                             Valdez, Harlan Jr.  \n                                           845-41-54-4\n                                             February 12, 2106 \nHar is a 43 year old 6\' 214 pound gentleman who is referred for\nconsultation by Dr. Harlan Oneil.  About a week ago he slipped on\nthe driveway at home and sustained an injury to his left ankle. \nHe was seen at Tri-City Hospital and was told he had a\nfracture.  He was placed in an air splint and advised to be\npartial weight bearing, and he is using a cane.  He is here for\nroutine follow-up. \nPast medical history is notable for no ankle injuries previously. \nHe has a history of diabetes and sleep apnea.  He takes Prozac,\nCardizem, Glucophage and Amaryl.  He is also followed by Dr. Harold\nNutter for an arrhythmia.  He does not smoke.  He drinks\nminimally.  He is a 

In [2]:
raw = pd.read_csv("clean_report_text_sent.csv")


In [3]:
raw.report_text_sent[0],raw.report_text_sent[1],raw.report_text_sent[2],raw.report_text_sent[3]

('Record date   Campbell Orthopedic Associates',
 'Madera Circle Omak GA',
 'Habib Valenzuela M.D.',
 'Valdez Harlan Jr.')

In [ ]:
text = 'Record date   Campbell Orthopedic Associates'


In [5]:
import spacy

nlp = spacy.load("en_core_web_md")

In [6]:
chunks = []
doc = nlp("Why does it hurt to run very fast.")

for token in doc:
    token_index = token.i  # this is the token index in the document
    prev_token = doc[token_index + 2]  # the previous token in the document
    print("Current Token:", token.text, "- Prev Token:", prev_token.text)
            # and so on

Current Token: Why - Prev Token: it
Current Token: does - Prev Token: hurt
Current Token: it - Prev Token: to
Current Token: hurt - Prev Token: run
Current Token: to - Prev Token: very
Current Token: run - Prev Token: fast
Current Token: very - Prev Token: .


IndexError: [E040] Attempt to access token at 9, max length 9.

In [7]:
text = 'Record date   Campbell Orthopedic Associates'
doc = nlp(text)
for token in doc:
    print(token.text, token.pos, token.pos_)

Record 92 NOUN
date 92 NOUN
   103 SPACE
Campbell 96 PROPN
Orthopedic 96 PROPN
Associates 96 PROPN


In [8]:
import spacy
from nltk import Tree

document_string = "The Waystone Inn lay in silence, and it was a silence of three parts. "

doc = nlp(document_string)

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_


[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]



                           lay                            
  __________________________|_________                     
 |   |       |              |        was                  
 |   |       |              |      ___|_________           
 |   |       |              |     |   |      silence      
 |   |       |              |     |   |    _____|______    
 |   |       |              |     |   |   |            of 
 |   |       |              |     |   |   |            |   
 |   |      Inn             in    |   |   |          parts
 |   |    ___|_____         |     |   |   |            |   
 ,  and The     Waystone silence  it  .   a          three



[None]

In [9]:
def print_table(rows, padding=0):
    """ Print `rows` with content-based column widths. """
    col_widths = [
        max(len(str(value)) for value in col) + padding
        for col in zip(*rows)
    ]
    total_width = sum(col_widths) + len(col_widths) - 1
    fmt = ' '.join('%%-%ds' % width for width in col_widths)
    print(fmt % tuple(rows[0]))
    print('~' * total_width)
    for row in rows[1:]:
        print(fmt % tuple(row))

In [10]:
rows = [['Chunk', '.root', 'root.dep_', '.root.head']]
for chunk in doc.noun_chunks:
    rows.append([
        chunk,            # A Span object with the full phrase.
        chunk.root,       # The key Token within this phrase.
        chunk.root.dep_,  # The grammatical role of this phrase.
        chunk.root.head   # The grammatical parent Token.
    ])
print_table(rows, padding=4)

Chunk                .root       root.dep_     .root.head    
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The Waystone Inn     Inn         nsubj         lay           
silence              silence     pobj          in            
it                   it          nsubj         was           
a silence            silence     attr          was           
three parts          parts       pobj          of            


In [23]:
def get_Parent_dep(doc, token_text):
    for chunk in doc.noun_chunks:
        if (str(chunk)).find(token_text) != -1:
            return chunk, '-', token_text, '-', chunk.root.head.text
        

In [24]:
document_string = "The Waystone Inn lay in silence, and it was a silence of three parts. "

doc = nlp(document_string)

for tkn in doc:
    print(tkn.text, get_Parent_dep(doc, tkn.text))

The (The Waystone Inn, '-', 'The', '-', 'lay')
Waystone (The Waystone Inn, '-', 'Waystone', '-', 'lay')
Inn (The Waystone Inn, '-', 'Inn', '-', 'lay')
lay None
in None
silence (silence, '-', 'silence', '-', 'in')
, None
and None
it (it, '-', 'it', '-', 'was')
was None
a (The Waystone Inn, '-', 'a', '-', 'lay')
silence (silence, '-', 'silence', '-', 'in')
of None
three (three parts, '-', 'three', '-', 'of')
parts (three parts, '-', 'parts', '-', 'of')
. None


In [9]:
sentences = ["The Waystone Inn lay in silence, and it was a silence of three parts. ","this is a new sentence"]
sentence = sentences[0]
window = 3
n = len(sentence)

# Previous unigrams
for i in range(n):
    end = min(i, window)
    unigrams = sentence[i - end:i]
    print(unigrams)
    #for j, u in enumerate(unigrams):
        #print(u)
        #features_list[i][('prev_unigrams-%d' % j, u)] = 1

# Next     unigrams
for i in range(n):
    end = min(i + window, n - 1)
    unigrams = sentence[i + 1:end + 1]
    #for j, u in enumerate(unigrams):
        #print(u)
        #features_list[i][('next_unigrams-%d' % j, u)] = 1


T
Th
The
he 
e W
 Wa
Way
ays
yst
sto
ton
one
ne 
e I
 In
Inn
nn 
n l
 la
lay
ay 
y i
 in
in 
n s
 si
sil
ile
len
enc
nce
ce,
e, 
, a
 an
and
nd 
d i
 it
it 
t w
 wa
was
as 
s a
 a 
a s
 si
sil
ile
len
enc
nce
ce 
e o
 of
of 
f t
 th
thr
hre
ree
ee 
e p
 pa
par
art
rts
ts.


In [3]:

def IOB_prose_features(sentence):
    """
    IOB_prose_features

    @param sentence. A list of strings
    @return          A list of dictionaries of features

    """
    features_list = []

    # Feature: Bag of Words unigram conext (window=3)
    if 'unigram_context' in enabled_IOB_prose_sentence_features:
        window = 3
        n = len(sentence)

        # Previous unigrams
        for i in range(n):
            end = min(i, window)
            unigrams = sentence[i - end:i]
            for j, u in enumerate(unigrams):
                features_list[i][('prev_unigrams-%d' % j, u)] = 1

        # Next     unigrams
        for i in range(n):
            end = min(i + window, n - 1)
            unigrams = sentence[i + 1:end + 1]
            for j, u in enumerate(unigrams):
                features_list[i][('next_unigrams-%d' % j, u)] = 1

    # Only POS tag once
    if 'pos' in enabled_IOB_prose_sentence_features:
        pos_tagged = nltk_tagger.tag(sentence)

    # Allow for particular features to be enabled
    for feature in enabled_IOB_prose_sentence_features:

        # Feature: Part of Speech
        if feature == 'pos':
            for (i, (_, pos)) in enumerate(pos_tagged):
                features_list[i].update({('pos', pos): 1})

        # Feature: POS context
        if 'pos_context' in enabled_IOB_prose_sentence_features:
            window = 3
            n = len(sentence)

            # Previous POS
            for i in range(n):
                end = min(i, window)
                for j, p in enumerate(pos_tagged[i - end:i]):
                    pos = p[1]
                    features_list[i][('prev_pos_context-%d' % j, pos)] = 1

            # Next POS
            for i in range(n):
                end = min(i + window, n - 1)
                for j, p in enumerate(pos_tagged[i + 1:i + end + 1]):
                    pos = p[1]
                    features_list[i][('prev_pos_context-%d' % j, pos)] = 1

        # GENIA features
        # if (feature == 'GENIA') and enabled['GENIA']:
        #
        #     # Get GENIA features
        #     genia_feat_list = feat_genia.features(sentence)
        #
        #     '''
        #     print '\t', sentence
        #
        #     print '\n\n'
        #     for gf in genia_feat_list:
        #         print '\t', gf
        #         print
        #     print '\n\n'
        #     '''
        #
        #     for i,feat_dict in enumerate(genia_feat_list):
        #         features_list[i].update(feat_dict)

        # Feature: UMLS Word Features (only use prose ones)
        if (feature == "UMLS"):
            umls_features = feat_umls.IOB_prose_features(sentence)
            for i in range(len(sentence)):
                features_list[i].update(umls_features[i])

    # Used for 'prev' and 'next' features
    ngram_features = [{} for i in range(len(features_list))]
    if "prev" in enabled_IOB_prose_sentence_features:
        prev = lambda f: {("prev_" + k[0], k[1]): v for k, v in f.items()}
        prev_list = map(prev, features_list)
        for i in range(len(features_list)):
            if i == 0:
                ngram_features[i][("prev", "*")] = 1
            else:
                ngram_features[i].update(prev_list[i - 1])

    if "prev2" in enabled_IOB_prose_sentence_features:
        prev2 = lambda f: {("prev2_" + k[0], k[1]): v / 2.0 for k, v in f.items()}
        prev_list = map(prev2, features_list)
        for i in range(len(features_list)):
            if i == 0:
                ngram_features[i][("prev2", "*")] = 1
            elif i == 1:
                ngram_features[i][("prev2", "*")] = 1
            else:
                ngram_features[i].update(prev_list[i - 2])

    if "next" in enabled_IOB_prose_sentence_features:
        next = lambda f: {("next_" + k[0], k[1]): v for k, v in f.items()}
        next_list = map(next, features_list)
        for i in range(len(features_list)):
            if i < len(features_list) - 1:
                ngram_features[i].update(next_list[i + 1])
            else:
                ngram_features[i][("next", "*")] = 1

    if "next2" in enabled_IOB_prose_sentence_features:
        next2 = lambda f: {("next2_" + k[0], k[1]): v / 2.0 for k, v in f.items()}
        next_list = map(next2, features_list)
        for i in range(len(features_list)):
            if i < len(features_list) - 2:
                ngram_features[i].update(next_list[i + 2])
            elif i == len(features_list) - 2:
                ngram_features[i][("next2", "**")] = 1
            else:
                ngram_features[i][("next2", "*")] = 1

    merged = lambda d1, d2: dict(d1.items() + d2.items())
    features_list = [merged(features_list[i], ngram_features[i])
                     for i in range(len(features_list))]

    '''
    for f in features_list:
        print sorted(f.items())
        print
    print '\n\n\n'
    '''

    return features_list


In [2]:
text = "The Waystone Inn lay in silence, and it was a silence of three parts. "
IOB_prose_features(text)

NameError: name 'feat_word' is not defined